# Employee Information Retrieval using MCP

This notebook demonstrates how to use the Model Context Protocol (MCP) to retrieve employee information from a dedicated server. We'll connect to an Employee Information tool server and use a language model to query employee data.

## Key Components

1. **Azure OpenAI Integration**: Using Azure's OpenAI service to power our agent
2. **MCP Client Setup**: Connecting to the Employee Information tool server
3. **Agent Creation**: Building a ReAct agent that can use the MCP tools
4. **Query Execution**: Retrieving specific employee information

## Required Library Imports

The following cell imports all necessary libraries for working with MCP, Azure OpenAI, and LangGraph.

In [2]:
from langchain_openai import AzureChatOpenAI
import random
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
import time
from langchain_mcp_adapters.client import MultiServerMCPClient

## Azure OpenAI Configuration

Setting up the Azure OpenAI client with appropriate API credentials and parameters.

In [ ]:
AZURE_API_VERSION = "2024-10-21"
AZURE_ENDPOINT = "Azure_End_Point"
AZURE_API_KEY = "<Azure_API_KEy>"

def llm():
    model = AzureChatOpenAI(
        api_version= AZURE_API_VERSION,
        azure_endpoint= AZURE_ENDPOINT,
        api_key= AZURE_API_KEY,
        azure_deployment="gpt-4o",
        verbose=True
    )
    return model

azure_api_client = llm()
# Setting up the Azure OpenAI model
# Note: In a production environment, store these keys securely, not directly in code

## Connecting to the Employee Information MCP Server

In this section, we establish a connection to the Employee Information tool server using the MCP client. This server provides tools that can retrieve employee data.

In [ ]:
# Connecting to the Employee Information server using MCP
# The server should be running on localhost:8001 with SSE transport
async with MultiServerMCPClient(
    {
        "Employee_Info_tool_server": {
            "url": "http://localhost:8001/sse",
            "transport": "sse",
        }
    }
) as client:
    tools = client.get_tools()

## Displaying Available Tools

Let's examine the available tools provided by the Employee Information server.

In [5]:
tools

[StructuredTool(name='get_employee_supervisor', description="\n    Returns the supervisor for a given employee ID.\n\n    Args:\n        employee_ID (str): The unique identifier of an employee.\n\n    Returns:\n        str: Supervisor's name associated with the given employee ID.\n    ", args_schema={'properties': {'employee_ID': {'title': 'Employee Id', 'type': 'string'}}, 'required': ['employee_ID'], 'title': 'get_employee_supervisorArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x121f05300>),
 StructuredTool(name='get_employee_location', description='\n    Returns the location of a given employee.\n\n    Args:\n        employee_name (str): The name of the employee.\n\n    Returns:\n        str: Location where the employee is based.\n    ', args_schema={'properties': {'employee_name': {'title': 'Employee Name', 'type': 'string'}}, 'required': ['employee_name'], 'title': 'get_employee_

## Creating an Agent and Querying Employee Information

Now we'll create a ReAct agent that uses the Azure OpenAI model and the MCP tools to query information about an employee named David.

In [7]:
async with MultiServerMCPClient(
    {
        "Employee_Info_tool_server": {
            "url": "http://localhost:8001/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(azure_api_client, client.get_tools())
    response = await agent.ainvoke({"messages": "what is the location of David"})

## Agent Response

Examining the response from our agent after it has queried the employee information.

In [8]:
response

{'messages': [HumanMessage(content='what is the location of David', additional_kwargs={}, response_metadata={}, id='f29295bc-23c9-4f97-853b-288105230b39'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E8rEoI2NdcfyCxO6EYA9QQvz', 'function': {'arguments': '{"employee_name":"David"}', 'name': 'get_employee_location'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 233, 'total_tokens': 250, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BicbJhxIKnhv4OR6IDwBbOtqPspjs', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm'

## Try Different Queries

Let's try a few more queries to test the capabilities of our employee information system.

In [ ]:
# Example of querying information about another employee
async with MultiServerMCPClient(
    {
        "Employee_Info_tool_server": {
            "url": "http://localhost:8001/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(azure_api_client, client.get_tools())
    # You can modify this query to ask different questions
    response = await agent.ainvoke({"messages": "What is the department of Sarah?"})
    
response

## Conclusion

In this notebook, we've demonstrated how to use the Model Context Protocol (MCP) to create an agent that can interact with an Employee Information tool server. This architecture allows for:

1. **Modular Tool Development**: The employee information tools can be developed and maintained separately from the agent
2. **Scalability**: Additional tool servers can be easily connected to the same agent
3. **Flexibility**: The agent can dynamically use the tools provided by the server without needing to be reconfigured

This approach is particularly powerful for building enterprise applications where different teams might maintain different services and data sources.

## References

1. [LangChain MCP Adapters](https://github.com/langchain-ai/langchain-mcp-adapters)
2. [Model Context Protocol - Tools](https://modelcontextprotocol.io/docs/concepts/tools)
3. [LangGraph MCP Agents Hands-On](https://github.com/teddynote-lab/langgraph-mcp-agents/blob/master/MCP-HandsOn-ENG.ipynb)